# Financial News Dataset - Exploratory Data Analysis (EDA)
# ================================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
from collections import Counter
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


### ================================================================
### 1. DATA LOADING AND INITIAL EXPLORATION
### ================================================================

In [ ]:
def load_and_explore_data(file_path):
    """Load the dataset and perform initial exploration"""
    print("🔍 LOADING AND EXPLORING DATA")
    print("=" * 50)
    
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Basic information
    print(f"Dataset Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nFirst 5 rows:")
    print(df.head())
    
    print("\nDataset Info:")
    print(df.info())
    
    print("\nMissing Values:")
    print(df.isnull().sum())
    
    print("\nBasic Statistics:")
    print(df.describe(include='all'))
    
    return df

df = load_and_explore_data('../data/raw_analyst_ratings/raw_analyst_ratings.csv')
print(f"Dataset Shape: {df.shape}")

### ================================================================
### 2. DATA PREPROCESSING
### ================================================================

In [ ]:
def preprocess_data(df):
    """Clean and preprocess the data"""
    print("\n🔧 DATA PREPROCESSING")
    print("=" * 50)
    
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    # Convert date column to datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')
    
    df_clean['year'] = df_clean['date'].dt.year
    df_clean['month'] = df_clean['date'].dt.month
    df_clean['day'] = df_clean['date'].dt.day
    df_clean['hour'] = df_clean['date'].dt.hour
    df_clean['day_of_week'] = df_clean['date'].dt.day_name()
    df_clean['is_weekend'] = df_clean['date'].dt.weekday >= 5
    
    df_clean['headline_length'] = df_clean['headline'].str.len()
    
    df_clean['domain'] = df_clean['url'].str.extract(r'https?://(?:www\.)?([^/]+)')
    
    df_clean['is_email_publisher'] = df_clean['publisher'].str.contains('@', na=False)
    df_clean['publisher_domain'] = df_clean['publisher'].str.extract(r'@([^.]+\.[^.]+)').fillna('')
    
    print("Preprocessing completed!")
    print(f"Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")
    
    return df_clean

df_processed = preprocess_data(df)

### ================================================================
### 3. DESCRIPTIVE STATISTICS
### ================================================================

In [ ]:
def descriptive_statistics(df):
    """Generate comprehensive descriptive statistics"""
    print("\n📈 DESCRIPTIVE STATISTICS")
    print("=" * 50)
    
    total_articles = len(df)
    unique_publishers = df['publisher'].nunique()
    unique_stocks = df['stock'].nunique()
    unique_domains = df['domain'].nunique()
    
    # Headline length statistics
    headline_stats = df['headline_length'].describe()
    
    print(f"📊 BASIC METRICS:")
    print(f"   Total Articles: {total_articles:,}")
    print(f"   Unique Publishers: {unique_publishers:,}")
    print(f"   Unique Stocks: {unique_stocks:,}")
    print(f"   Unique Domains: {unique_domains:,}")
    print(f"   Email Publishers: {df['is_email_publisher'].sum():,}")
    
    print(f"\n📝 HEADLINE LENGTH STATISTICS:")
    for stat, value in headline_stats.items():
        print(f"   {stat.title()}: {value:.1f} characters")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    axes[0, 0].hist(df['headline_length'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribution of Headline Lengths')
    axes[0, 0].set_xlabel('Headline Length (characters)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].axvline(headline_stats['mean'], color='red', linestyle='--', label=f'Mean: {headline_stats["mean"]:.1f}')
    axes[0, 0].legend()
    
    # Top publishers
    top_publishers = df['publisher'].value_counts().head(10)
    axes[0, 1].barh(range(len(top_publishers)), top_publishers.values)
    axes[0, 1].set_yticks(range(len(top_publishers)))
    axes[0, 1].set_yticklabels(top_publishers.index)
    axes[0, 1].set_title('Top 10 Publishers by Article Count')
    axes[0, 1].set_xlabel('Number of Articles')
    
    # Stock distribution
    stock_counts = df['stock'].value_counts().head(10)
    axes[1, 0].pie(stock_counts.values, labels=stock_counts.index, autopct='%1.1f%%', startangle=90)
    axes[1, 0].set_title('Distribution of Articles by Stock')
    
    # Email vs Non-email publishers
    email_counts = df['is_email_publisher'].value_counts()
    axes[1, 1].bar(['Non-Email', 'Email'], email_counts.values, color=['lightcoral', 'lightblue'])
    axes[1, 1].set_title('Email vs Non-Email Publishers')
    axes[1, 1].set_ylabel('Number of Articles')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'total_articles': total_articles,
        'unique_publishers': unique_publishers,
        'headline_stats': headline_stats,
        'top_publishers': top_publishers
    }

desc_stats = descriptive_statistics(df_processed)

### ================================================================
### 4. TIME SERIES ANALYSIS
### ================================================================

In [ ]:
def time_series_analysis(df):
    """Perform comprehensive time series analysis"""
    print("\n⏰ TIME SERIES ANALYSIS")
    print("=" * 50)
    
    daily_counts = df.groupby(df['date'].dt.date).size().reset_index()
    daily_counts.columns = ['date', 'count']
    daily_counts['date'] = pd.to_datetime(daily_counts['date'])
    
    hourly_counts = df.groupby('hour').size()
    
    # Day of week distribution
    dow_counts = df.groupby('day_of_week').size()
    dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dow_counts = dow_counts.reindex(dow_order, fill_value=0)
    
    # Monthly trends
    monthly_counts = df.groupby([df['date'].dt.year, df['date'].dt.month]).size().reset_index()
    monthly_counts['date'] = pd.to_datetime(monthly_counts[['year', 'month']].assign(day=1))
    
    print(f"📅 TIME SERIES INSIGHTS:")
    print(f"   Date Range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"   Most Active Day: {dow_counts.idxmax()} ({dow_counts.max()} articles)")
    print(f"   Most Active Hour: {hourly_counts.idxmax()}:00 ({hourly_counts.max()} articles)")
    print(f"   Weekend Articles: {df['is_weekend'].sum()} ({df['is_weekend'].mean()*100:.1f}%)")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Daily publication frequency
    axes[0, 0].plot(daily_counts['date'], daily_counts['count'], marker='o', linewidth=2, markersize=4)
    axes[0, 0].set_title('Daily Publication Frequency')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Hourly distribution
    axes[0, 1].bar(hourly_counts.index, hourly_counts.values, color='lightgreen', alpha=0.7)
    axes[0, 1].set_title('Publication Distribution by Hour of Day')
    axes[0, 1].set_xlabel('Hour of Day')
    axes[0, 1].set_ylabel('Number of Articles')
    axes[0, 1].set_xticks(range(0, 24, 2))
    
    # Day of week distribution
    axes[1, 0].bar(range(len(dow_counts)), dow_counts.values, color='orange', alpha=0.7)
    axes[1, 0].set_title('Publication Distribution by Day of Week')
    axes[1, 0].set_xlabel('Day of Week')
    axes[1, 0].set_ylabel('Number of Articles')
    axes[1, 0].set_xticks(range(len(dow_counts)))
    axes[1, 0].set_xticklabels([day[:3] for day in dow_order], rotation=45)
    
    # Monthly trends (if applicable)
    if len(monthly_counts) > 1:
        axes[1, 1].plot(monthly_counts['date'], monthly_counts[0], marker='s', linewidth=2, markersize=6, color='purple')
        axes[1, 1].set_title('Monthly Publication Trends')
        axes[1, 1].set_xlabel('Month')
        axes[1, 1].set_ylabel('Number of Articles')
        axes[1, 1].tick_params(axis='x', rotation=45)
    else:
        axes[1, 1].text(0.5, 0.5, 'Insufficient data\nfor monthly trends', 
                       ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
        axes[1, 1].set_title('Monthly Publication Trends')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'daily_counts': daily_counts,
        'hourly_counts': hourly_counts,
        'dow_counts': dow_counts,
        'peak_hour': hourly_counts.idxmax(),
        'peak_day': dow_counts.idxmax()
    }

time_analysis = time_series_analysis(df_processed)

### ================================================================
### 5. TEXT ANALYSIS & TOPIC MODELING
### ================================================================

In [ ]:
def text_analysis(df):
    """Perform comprehensive text analysis on headlines"""
    print("\n📝 TEXT ANALYSIS & TOPIC MODELING")
    print("=" * 50)
    
    # Combine all headlines
    all_text = ' '.join(df['headline'].astype(str))
    
    # Common financial keywords to look for
    financial_keywords = [
        'price target', 'target price', 'maintains', 'raises', 'lowers', 'upgrades', 'downgrades',
        'earnings', 'revenue', 'eps', 'quarterly', 'annual', 'q1', 'q2', 'q3', 'q4',
        'fda approval', 'fda', 'approval', 'merger', 'acquisition', 'dividend', 'stock split',
        'buyback', 'shares', 'trading', 'higher', 'lower', 'up', 'down', 'bull', 'bear',
        'market', 'stock', 'analyst', 'rating', 'recommendation', 'buy', 'sell', 'hold',
        'neutral', 'overweight', 'underweight', 'outperform', 'underperform'
    ]
    
    keyword_counts = {}
    for keyword in financial_keywords:
        count = all_text.lower().count(keyword.lower())
        if count > 0:
            keyword_counts[keyword] = count
    
    stop_words = set(['the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should'])
    
    words = re.findall(r'\b[a-zA-Z]{3,}\b', all_text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    word_counts = Counter(filtered_words)
    
    print(f"🔤 TEXT ANALYSIS RESULTS:")
    print(f"   Total Words: {len(words):,}")
    print(f"   Unique Words: {len(set(words)):,}")
    print(f"   Financial Keywords Found: {len(keyword_counts)}")
    
    print(f"\n🎯 TOP FINANCIAL KEYWORDS:")
    sorted_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    for keyword, count in sorted_keywords:
        print(f"   '{keyword}': {count} occurrences")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Top words
    top_words = dict(word_counts.most_common(15))
    axes[0, 0].barh(list(top_words.keys()), list(top_words.values()))
    axes[0, 0].set_title('Top 15 Most Frequent Words')
    axes[0, 0].set_xlabel('Frequency')
    
    # Financial keywords
    if sorted_keywords:
        kw_names, kw_counts = zip(*sorted_keywords[:10])
        axes[0, 1].bar(range(len(kw_names)), kw_counts, color='lightcoral')
        axes[0, 1].set_title('Top Financial Keywords')
        axes[0, 1].set_xlabel('Keywords')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].set_xticks(range(len(kw_names)))
        axes[0, 1].set_xticklabels(kw_names, rotation=45, ha='right')
    
    # Word length distribution
    word_lengths = [len(word) for word in filtered_words]
    axes[1, 0].hist(word_lengths, bins=15, alpha=0.7, color='lightblue', edgecolor='black')
    axes[1, 0].set_title('Distribution of Word Lengths')
    axes[1, 0].set_xlabel('Word Length (characters)')
    axes[1, 0].set_ylabel('Frequency')
    
    # Create word cloud
    if len(filtered_words) > 10:
        try:
            wordcloud = WordCloud(width=400, height=300, background_color='white', 
                                max_words=50, colormap='viridis').generate(' '.join(filtered_words))
            axes[1, 1].imshow(wordcloud, interpolation='bilinear')
            axes[1, 1].axis('off')
            axes[1, 1].set_title('Word Cloud of Headlines')
        except:
            axes[1, 1].text(0.5, 0.5, 'Word Cloud\nNot Available', 
                           ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'word_counts': word_counts,
        'keyword_counts': keyword_counts,
        'top_words': top_words,
        'total_words': len(words),
        'unique_words': len(set(words))
    }

text_results = text_analysis(df_processed)

### ================================================================
### 6. PUBLISHER ANALYSIS
### ================================================================

In [ ]:
def publisher_analysis(df):
    """Analyze publisher patterns and characteristics"""
    print("\n👥 PUBLISHER ANALYSIS")
    print("=" * 50)
    
    publisher_stats = df['publisher'].value_counts()
    email_publishers = df[df['is_email_publisher'] == True]['publisher'].value_counts()
    domain_stats = df[df['publisher_domain'] != '']['publisher_domain'].value_counts()
    
    publisher_activity = df.groupby('publisher').agg({
        'headline_length': ['mean', 'std'],
        'date': ['min', 'max', 'count'],
        'stock': 'nunique'
    }).round(2)
    
    publisher_activity.columns = ['avg_headline_length', 'std_headline_length', 
                                'first_article', 'last_article', 'total_articles', 'unique_stocks']
    
    print(f"📊 PUBLISHER STATISTICS:")
    print(f"   Total Publishers: {len(publisher_stats)}")
    print(f"   Email-based Publishers: {len(email_publishers)}")
    print(f"   Most Active Publisher: {publisher_stats.index[0]} ({publisher_stats.iloc[0]} articles)")
    
    if len(domain_stats) > 0:
        print(f"   Most Common Domain: {domain_stats.index[0]} ({domain_stats.iloc[0]} articles)")
    
    print(f"\n🏆 TOP 5 PUBLISHERS:")
    for i, (publisher, count) in enumerate(publisher_stats.head().items(), 1):
        print(f"   {i}. {publisher}: {count} articles")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    top_publishers = publisher_stats.head(10)
    axes[0, 0].barh(range(len(top_publishers)), top_publishers.values)
    axes[0, 0].set_yticks(range(len(top_publishers)))
    axes[0, 0].set_yticklabels(top_publishers.index)
    axes[0, 0].set_title('Top 10 Publishers by Article Count')
    axes[0, 0].set_xlabel('Number of Articles')
    
    email_dist = df['is_email_publisher'].value_counts()
    axes[0, 1].pie(email_dist.values, labels=['Non-Email', 'Email'], autopct='%1.1f%%', 
                   colors=['lightblue', 'lightcoral'], startangle=90)
    axes[0, 1].set_title('Email vs Non-Email Publishers')
    
    if len(publisher_activity) > 1:
        top_pub_headlines = publisher_activity.head(8)['avg_headline_length']
        axes[1, 0].bar(range(len(top_pub_headlines)), top_pub_headlines.values, color='lightgreen')
        axes[1, 0].set_title('Average Headline Length by Top Publishers')
        axes[1, 0].set_xlabel('Publishers')
        axes[1, 0].set_ylabel('Average Headline Length')
        axes[1, 0].set_xticks(range(len(top_pub_headlines)))
        axes[1, 0].set_xticklabels(top_pub_headlines.index, rotation=45, ha='right')
    
    if len(domain_stats) > 0:
        axes[1, 1].bar(range(len(domain_stats)), domain_stats.values, color='orange')
        axes[1, 1].set_title('Articles by Email Domain')
        axes[1, 1].set_xlabel('Email Domains')
        axes[1, 1].set_ylabel('Number of Articles')
        axes[1, 1].set_xticks(range(len(domain_stats)))
        axes[1, 1].set_xticklabels(domain_stats.index, rotation=45, ha='right')
    else:
        axes[1, 1].text(0.5, 0.5, 'No Email Domains\nFound', 
                       ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'publisher_stats': publisher_stats,
        'email_publishers': email_publishers,
        'domain_stats': domain_stats,
        'publisher_activity': publisher_activity
    }

publisher_results = publisher_analysis(df_processed)

### ================================================================
### 7. ADVANCED ANALYSIS & INSIGHTS
### ================================================================

In [ ]:
def advanced_insights(df):
    """Generate advanced insights and correlations"""
    print("\n🎯 ADVANCED INSIGHTS & CORRELATIONS")
    print("=" * 50)
    
    # Correlation analysis
    numeric_cols = ['headline_length', 'hour', 'day', 'month']
    correlation_matrix = df[numeric_cols].corr()
    
    publisher_stock_diversity = df.groupby('publisher')['stock'].nunique().sort_values(ascending=False)
    
    # Time-based patterns
    hour_publisher = pd.crosstab(df['hour'], df['publisher'])
    
    # Peak activity analysis
    peak_hours = df['hour'].value_counts().head(3)
    peak_days = df['day_of_week'].value_counts().head(3)
    
    print(f"🔍 ADVANCED INSIGHTS:")
    print(f"   Most Diverse Publisher: {publisher_stock_diversity.index[0]} ({publisher_stock_diversity.iloc[0]} stocks)")
    print(f"   Peak Publishing Hours: {', '.join([f'{h}:00' for h in peak_hours.index])}")
    print(f"   Peak Publishing Days: {', '.join(peak_days.index)}")
    
    hourly_headline_length = df.groupby('hour')['headline_length'].mean()
    
    print(f"\n📊 HEADLINE PATTERNS:")
    longest_avg_hour = hourly_headline_length.idxmax()
    shortest_avg_hour = hourly_headline_length.idxmin()
    print(f"   Longest headlines published at: {longest_avg_hour}:00 (avg: {hourly_headline_length[longest_avg_hour]:.1f} chars)")
    print(f"   Shortest headlines published at: {shortest_avg_hour}:00 (avg: {hourly_headline_length[shortest_avg_hour]:.1f} chars)")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Correlation heatmap
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[0, 0])
    axes[0, 0].set_title('Correlation Matrix of Numeric Variables')
    
    # Publisher stock diversity
    top_diverse = publisher_stock_diversity.head(8)
    axes[0, 1].bar(range(len(top_diverse)), top_diverse.values, color='lightpink')
    axes[0, 1].set_title('Publisher Stock Diversity')
    axes[0, 1].set_xlabel('Publishers')
    axes[0, 1].set_ylabel('Number of Unique Stocks')
    axes[0, 1].set_xticks(range(len(top_diverse)))
    axes[0, 1].set_xticklabels(top_diverse.index, rotation=45, ha='right')
    
    # Hourly headline length trends
    axes[1, 0].plot(hourly_headline_length.index, hourly_headline_length.values, 
                   marker='o', linewidth=2, markersize=6, color='green')
    axes[1, 0].set_title('Average Headline Length by Hour')
    axes[1, 0].set_xlabel('Hour of Day')
    axes[1, 0].set_ylabel('Average Headline Length')
    axes[1, 0].set_xticks(range(0, 24, 2))
    axes[1, 0].grid(True, alpha=0.3)
    
    # Publication frequency heatmap (hour vs day of week)
    if len(df) > 10:  # Only if we have sufficient data
        hour_dow = pd.crosstab(df['hour'], df['day_of_week'])
        
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        hour_dow = hour_dow.reindex(columns=[day for day in day_order if day in hour_dow.columns])
        
        sns.heatmap(hour_dow, cmap='YlOrRd', ax=axes[1, 1], cbar_kws={'label': 'Number of Articles'})
        axes[1, 1].set_title('Publication Heatmap: Hour vs Day of Week')
        axes[1, 1].set_xlabel('Day of Week')
        axes[1, 1].set_ylabel('Hour of Day')
    else:
        axes[1, 1].text(0.5, 0.5, 'Insufficient Data\nfor Heatmap', 
                       ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'correlation_matrix': correlation_matrix,
        'publisher_diversity': publisher_stock_diversity,
        'peak_hours': peak_hours,
        'hourly_headline_length': hourly_headline_length
    }

advanced_results = advanced_insights(df_processed)